In [58]:
import xml.etree.ElementTree as ET
import pandas as pd

In [2]:
tree = ET.parse('./scratch/kml.php')
root = tree.getroot()

In [26]:
ns = '{http://www.opengis.net/kml/2.2}'

In [71]:
out = []
for fol in root[0].findall(ns + 'Folder'):
    for mark in fol.findall(ns + 'Placemark'):
        out.append({
            'name': mark.find(ns + 'name').text,
            'desc': mark.find(ns + 'description').text,
            'coords': mark.find(ns + 'Point').find(ns + 'coordinates').text,
            'scale': mark.find(ns + 'Style').find(ns + 'IconStyle').find(ns + 'scale').text
        })

In [82]:
dta = pd.DataFrame.from_dict(out)
dta = dta[dta.name.str.contains('CZECH REPUBLIC')]

In [90]:
dta['x'] = dta.coords.apply(lambda x: float(x.split(',')[1]))
dta['y'] = dta.coords.apply(lambda x: float(x.split(',')[0]))

dta['mag'] = dta.name.apply(lambda x: float(x.split('M')[1].split(' - ')[0].replace('L ', '')))

In [96]:
dta['date'] = dta.desc.apply(lambda x: x.split('<b>')[1].split(' ')[0])

In [112]:
dta.scale = dta.scale.astype('float')

In [115]:
dta[dta.mag >= 3][['scale', 'x', 'y', 'mag', 'date']].to_json('./quakes.json', orient='index')